In [1]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash 
pwd

/content/Template_Project


In [120]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)
from core.dl_framework.utils import get_specific_history
from core.dl_framework.utils import plot_history, plot_history_all

In [ ]:
from core.dl_framework.learner import Learner
from core.dl_framework.callbacks import Recorder, Monitor, EarlyStopping, Checkpoints
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
from torchvision import transforms
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"], CNN=True)

train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, test_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)
learn = Learner(train_ds, setup_config)
learn.fit(20)

In [4]:
from pathlib import Path
from datetime import datetime
ckp_path = "/content/gdrive/MyDrive/Marcel_Moczarski/Data_Science/Google_Services/Local_Drive_Storage/Conda_Envs/Template_Project/checkpoints"

In [26]:
# sample = next(iter(test_dl))
# imgs, targets = sample

In [126]:
best_hist = get_specific_history(Path(setup_config["c_c_ckp_path"]+"/"+"2022-04-15"), "valid_loss", "csv", "all")
plot_history_all(best_hist)


[    epochs  valid_acc  valid_loss  train_loss
 0        1   0.912260    0.590546    1.222598
 1        2   0.944511    0.312849    0.435798
 2        3   0.956530    0.216802    0.271130
 3        4   0.963141    0.167376    0.200384
 4        5   0.968049    0.137054    0.159935
 5        6   0.971454    0.116566    0.133426
 6        7   0.974559    0.101837    0.114665
 7        8   0.977264    0.090799    0.100660
 8        9   0.979367    0.082228    0.089817
 9       10   0.980769    0.075381    0.081155
 10      11   0.982272    0.069765    0.074066]

In [13]:
[   epochs  valid_acc  valid_loss  train_loss
 0       1   0.901738    0.698227    1.291457
 1       2   0.939840    0.370269    0.507460
 2       3   0.954128    0.255636    0.309141
 3       4   0.960394    0.197812    0.225584
 4       5   0.965074    0.162826    0.178947
 5       6   0.967914    0.139301    0.148855
 6       7   0.970755    0.122444    0.127679]